In [1]:
import pandas as pd
import numpy as np
import os
import glob
from scipy import stats
from pandas.util.testing import assert_frame_equal

In [13]:
# input_dir = '/projects/ps-yeolab3/bay001/maps/current/se_peak/'
input_dir = '/home/bay001/projects/brian_rbpmaps_20180202/temporary_data/peak_bb/'

all_native = '204_01.basedon_204_01.peaks.l2inputnormnew.bed.compressed.bed.p3f3.bed.sorted.bed.bb.HepG2_native_cassette_exons_all.ip.raw_density.txt'
included = '204_01.basedon_204_01.peaks.l2inputnormnew.bed.compressed.bed.p3f3.bed.sorted.bed.bb.RBFOX2-BGHLV26-HepG2.set26-included-upon-knockdown.ip.raw_density.txt'
excluded = '204_01.basedon_204_01.peaks.l2inputnormnew.bed.compressed.bed.p3f3.bed.sorted.bed.bb.RBFOX2-BGHLV26-HepG2.set26-excluded-upon-knockdown.ip.raw_density.txt'

# let's be very explicit about the steps taken to get this histogram
all_hist = pd.read_table(os.path.join(input_dir, all_native), sep=',', index_col=0)
incl_hist = pd.read_table(os.path.join(input_dir, included), sep=',', index_col=0)
excl_hist = pd.read_table(os.path.join(input_dir, excluded), sep=',', index_col=0)

# "clean" the dataframe (ie. replace -1 with np.nans, as -1 indicates positions with sufficient overlap)
all_hist = all_hist.replace(-1, np.nan)
incl_hist = incl_hist.replace(-1, np.nan)
excl_hist = excl_hist.replace(-1, np.nan)

# sum the number of peak overlaps at each position
all_hist = all_hist.sum()
incl_hist = incl_hist.sum()
excl_hist = excl_hist.sum()

# transform to list to fit function params
all_hist = list(all_hist)
incl_hist = list(incl_hist)
excl_hist = list(excl_hist)

In [4]:
# get the number of events
all_n = pd.read_table(os.path.join(input_dir, all_native), sep=',', index_col=0).shape[0]
incl_n = pd.read_table(os.path.join(input_dir, included), sep=',', index_col=0).shape[0]
excl_n = pd.read_table(os.path.join(input_dir, excluded), sep=',', index_col=0).shape[0]

all_n = all_n
excl_n = [excl_n]*1400
incl_n = [incl_n]*1400

In [5]:
def calculate_fisher(hist, bg_hist, num_events, bg_num_events):
    p_values = []
    # bg_num_events = bg_matrix.shape[0]
    # bg_hist = list(bg_matrix.sum())
    for i in range(0, len(hist)):
        total_peak = hist[i]  # total peaks at position i
        total_without_peak = sum(num_events)/len(num_events) - total_peak
        bg_total_peak = bg_hist[i]
        bg_total_without_peak = bg_num_events - bg_total_peak
        contingency_table = [
            [total_peak, total_without_peak],
            [bg_total_peak, bg_total_without_peak]
        ]
        odds, p = stats.fisher_exact(contingency_table)
        p_values.append(-1 * np.log10(p))

    return p_values

pd.Series(calculate_fisher(incl_hist, all_hist, incl_n, all_n))

0      -0.000000
1      -0.000000
2      -0.000000
3      -0.000000
4      -0.000000
5      -0.000000
6      -0.000000
7      -0.000000
8      -0.000000
9      -0.000000
10     -0.000000
11     -0.000000
12     -0.000000
13     -0.000000
14     -0.000000
15     -0.000000
16     -0.000000
17     -0.000000
18     -0.000000
19     -0.000000
20     -0.000000
21     -0.000000
22     -0.000000
23     -0.000000
24     -0.000000
25     -0.000000
26     -0.000000
27     -0.000000
28     -0.000000
29     -0.000000
          ...   
1370   -0.000000
1371   -0.000000
1372   -0.000000
1373   -0.000000
1374   -0.000000
1375   -0.000000
1376   -0.000000
1377   -0.000000
1378   -0.000000
1379    1.229770
1380    1.229770
1381    1.229770
1382    1.229770
1383    1.229770
1384    1.229770
1385    1.229770
1386    1.229770
1387    1.229770
1388    1.229770
1389    1.229770
1390    1.229770
1391    0.941616
1392    0.941616
1393    0.941616
1394    0.941616
1395    0.941616
1396    0.941616
1397    0.9416

In [8]:
pd.read_table(
    os.path.join(
        input_dir, 
        '204_01.basedon_204_01.peaks.l2inputnormnew.bed.compressed.bed.p3f3.bed.sorted.bed.bb.RBFOX2-BGHLV26-HepG2.set26-included-upon-knockdown.pvalues.txt'
    ),
    names=['p']
)

,p
0,-0.000000
1,-0.000000
2,-0.000000
3,-0.000000
4,-0.000000
5,-0.000000
6,-0.000000
7,-0.000000
8,-0.000000
9,-0.000000
